# Using a Fork of on-chain state

A fork is when we pull state from a live contract (or contracts) into the local EVM. To use forks, you need to have 
access to a Ethereum RPC node or RPC service such as [Alchemy](https://www.alchemy.com/) or Infura. Touching the 
contract will pull information into our local EVM where we can `snapshot` the state and save it for use later.

At a minimum, a fork makes it easy to work with contracts that may have a complicated setup for deployment. Pulling a
contract through a fork will grab all the associated libraries, proxy contracts, etc...

See `gen_usdc_state.py` for the script used to generate the snapshot used in this example. 

**What we'll do:**
- Load the snaphot `usdc_snapshot.json` created by the `gen_usdc_state.py` script
- mint and burn some tokens on the USDC contract
- Check the totalSupply
  
You can learn more about the methods available on the USDC contract here: [Etherscan](https://etherscan.io/address/0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48#readProxyContract)

## Setup

First, we'll import the functionality we'll need from Simular, copy over some of the 
constants we need from the `gen_usdc_state.py` script, and load the snapshot.

In [41]:
from simular import (
    PyEvm,
    contract_from_inline_abi,
    create_account,
)

# USDC master minters address on chain
MM = "0xe982615d461dd5cd06575bbea87624fda4e3de17"

# The minter we created in the snapshot
MINTER = "0x7ac8a704a0dafcc12fc54679b940fc17de02950a"

# minter allowance (how much USDC they're allowed to mint)
# Remember USDC has 6 decimal places. So, 1e10 is 10,000 USDC
MINTER_ALLOWANCE = int(1e10)

# on-chain USDC contract address
# https://etherscan.io/address/0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
USDC = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"

# load snapshot from file
with open('./usdc_snapshot.json') as f:
    snapshot = f.read()

Now create an EVM and load the state

In [42]:
evm = PyEvm.from_snapshot(snapshot)

To call USDC, we need a contract. We'll specify the methods we want to use.  They are
part of the USDC contract.

In [43]:
contract_methods = [
"function isMinter(address) (bool)",
"function totalSupply() (uint256)",
"function mint(address, uint256) (bool)",
"function burn(uint256)",
"function transfer(address, uint256) (bool)",
"function minterAllowance(address) (uint256)",
"function balanceOf(address) (uint256)"
]

usdc = contract_from_inline_abi(evm, contract_methods)

We need to point the `usdc` contract to the correct address - which is the USDC
we forked.

In [44]:
usdc.at(USDC)

In the snapshot we provided the ability for our minter mint 10,000 USDC. Let's check it...

In [45]:
assert MINTER_ALLOWANCE == usdc.minterAllowance.call(MINTER)

Let's verify the no one has minted any USDC yet...

In [46]:
assert 0 == usdc.totalSupply.call()

Now, the minter will mint is allowance to himself.

In [47]:
# each mints 
tx1 = usdc.mint.transact(MINTER, MINTER_ALLOWANCE, caller=MINTER)
print(f"Gas Used: {tx1.gas_used}")


Gas Used: 83036


In [48]:
assert MINTER_ALLOWANCE == usdc.totalSupply.call()

Just for the heck of it, let's create another account (say Alice) and see if they 
can mint some tokens...

In [49]:
alice = create_account(evm)
try: 
    usdc.mint.transact(alice, int(1e6), caller=alice)    
except: 
    print("NOPE! Not allowed alice!")

NOPE! Not allowed alice!


In [50]:
# confirm supply hasn't changed
assert MINTER_ALLOWANCE == usdc.totalSupply.call()

The MINTER transfers 100 USDC to alice

In [51]:
usdc.transfer.transact(alice, int(100 * 10**6), caller=MINTER)

Let's check balances:

In [52]:
alice_amount = 100 * 10**6
assert usdc.balanceOf.call(alice) == alice_amount
assert usdc.balanceOf.call(MINTER) == MINTER_ALLOWANCE - alice_amount
assert MINTER_ALLOWANCE == usdc.totalSupply.call()

Finally, let burn 1000 USDC...

In [53]:
burn_amount = 1000 * 10**6
usdc.burn.transact(burn_amount, caller=MINTER)

assert usdc.totalSupply.call() == MINTER_ALLOWANCE - burn_amount 

And that's it.  Here we showed pulling contract code and state from the on-chain USDC contract, loading the snapshot, and interacting with the contract. 